In [1]:
import pandas as pd
from tensorflow.keras import layers, models
import pickle
import numpy as np

In [ ]:
df = pd.read_csv("./data/10k_users.csv")[["user_id", "anime_id", "rating"]]
df.head(10)

In [ ]:
len(df.user_id.unique())
len(df.anime_id.unique())

In [ ]:
user_item_matrix = df.groupby(['user_id', 'anime_id'])['rating'].mean().unstack(fill_value=0)
user_item_matrix

In [ ]:
user_item_matrix.to_pickle("useritem.pkl")

In [ ]:
user_item_matrix = pd.read_pickle("useritem.pkl")
user_item_matrix

In [ ]:
df = pd.DataFrame()

In [ ]:
#normalized_user_item_matrix = pd.DataFrame(index=user_item_matrix.index, columns=user_item_matrix.columns)

for idx in user_item_matrix.index:
    row_values = user_item_matrix.loc[idx]
    min_value = row_values.min()
    max_value = row_values.max()
    normalized_values = row_values.apply(lambda x: 0 if x == 0 else (x - min_value) / (max_value - min_value) * 2 - 1)
    user_item_matrix.loc[idx] = normalized_values
    if idx % 1000 == 0:
        print(idx)
normalized_user_item_matrix_array = user_item_matrix.to_numpy()
normalized_user_item_matrix_array

In [ ]:
user_item_matrix.to_pickle("scaled_item-user.pkl")

In [ ]:
user_item_matrix = pd.read_pickle("scaled_item-user.pkl")
user_item_matrix

In [ ]:
total_entries = user_item_matrix.size
zero_entries = (user_item_matrix == 0).sum().sum()

sparsity_percentage = (zero_entries / total_entries) * 100

print(f"Sparsity of the matrix: {sparsity_percentage:.2f}%")

In [ ]:
def remove_ratings(matrix, percentage=0.25):
    """
    Remove a specified percentage of non-zero ratings from each row of the user-item matrix.

    Parameters:
    - matrix: pandas DataFrame
        The user-item matrix.
    - percentage: float, optional (default=0.25)
        The percentage of non-zero ratings to remove.

    Returns:
    - modified_matrix: pandas DataFrame
        The modified user-item matrix.
    """
    for idx in matrix.index:
        row_values = matrix.loc[idx]

        non_zero_indices = row_values[row_values != 0].index

        num_ratings_to_remove = int(len(non_zero_indices) * percentage)
        ratings_to_remove = np.random.choice(non_zero_indices, size=num_ratings_to_remove, replace=False)

        matrix.loc[idx, ratings_to_remove] = 0

    return matrix

In [ ]:
user_item_matrix = remove_ratings(user_item_matrix, percentage=0.25)

In [ ]:
total_entries = user_item_matrix.size
zero_entries = (user_item_matrix == 0).sum().sum()

sparsity_percentage = (zero_entries / total_entries) * 100

print(f"Sparsity of the matrix: {sparsity_percentage:.2f}%")

In [ ]:
user_item_matrix.to_pickle("denoised_item-user.pkl")

In [ ]:
noisy_matrix = pd.read_pickle("scaled_item-user.pkl")
noisy_matrix

## Interesting part comes on

In [2]:
import tensorflow as tf

In [11]:
model = models.Sequential()
model.add(layers.Dense(4000, input_shape=(4697,) ,activation="tanh"))
model.add(layers.Dense(4000, activation="tanh"))
model.add(layers.Dense(4000, activation="tanh"))
model.add(layers.Dense(4697, activation="linear")) 
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 4000)              18792000  
                                                                 
 dense_5 (Dense)             (None, 4000)              16004000  
                                                                 
 dense_6 (Dense)             (None, 4000)              16004000  
                                                                 
 dense_7 (Dense)             (None, 4697)              18792697  
                                                                 
Total params: 69592697 (265.48 MB)
Trainable params: 69592697 (265.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [4]:
def custom_loss(y_true, y_pred):
    weights = tf.where(tf.equal(y_true, 0), 0.1, 1.0)
    loss = tf.reduce_sum(tf.square(y_true - y_pred) * weights) / tf.reduce_sum(weights)
    
    return loss

In [12]:
model.compile(optimizer='adam', loss=custom_loss)

In [ ]:
#model.compile(optimizer='adam', loss=custom_loss)

In [ ]:
user_item_matrix = pd.read_pickle("scaled_item-user.pkl").to_numpy()
user_item_matrix

In [6]:
noisy_matrix = pd.read_pickle("scaled_item-user.pkl").to_numpy()
noisy_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
def data_generator(matrix, batch_size):
    num_samples = matrix.shape[0]
    while True:
        indices = np.random.choice(num_samples, size=batch_size, replace=False)
        batch_data = matrix[indices, :]
        yield batch_data, batch_data

In [20]:
batch_size = 1000

In [21]:
model.fit(data_generator(noisy_matrix, batch_size), steps_per_epoch=len(noisy_matrix)//batch_size, epochs=50)

Epoch 1/50
325/325 [==============================] - 496s 2s/step - loss: 0.0162
Epoch 2/50
325/325 [==============================] - 484s 1s/step - loss: 0.0073
Epoch 3/50
325/325 [==============================] - 484s 1s/step - loss: 0.0046
Epoch 4/50
325/325 [==============================] - 483s 1s/step - loss: 0.0034
Epoch 5/50
325/325 [==============================] - 483s 1s/step - loss: 0.0026
Epoch 6/50
325/325 [==============================] - 484s 1s/step - loss: 0.0022
Epoch 7/50
325/325 [==============================] - 484s 1s/step - loss: 0.0019
Epoch 8/50
325/325 [==============================] - 484s 1s/step - loss: 0.0016
Epoch 9/50
325/325 [==============================] - 484s 1s/step - loss: 0.0016
Epoch 10/50
325/325 [==============================] - 483s 1s/step - loss: 0.0014
Epoch 11/50
325/325 [==============================] - 483s 1s/step - loss: 0.0015
Epoch 12/50
325/325 [==============================] - 483s 1s/step - loss: 0.0015
Epoch 13/50
3

In [22]:
model.save_weights("./")

In [32]:
model.save("./7hours.keras")

In [24]:
noisy_matrix[0]

array([0., 0., 0., ..., 0., 0., 0.])

In [26]:
reconstructed_data = model.predict(noisy_matrix)

10173/10173 [==============================] - 248s 24ms/step


In [36]:
reconstructed_data.shape

(325509, 4697)

In [33]:
type(reconstructed_data)

numpy.ndarray

In [34]:
np.save("reconstruction.npy", reconstructed_data)

In [35]:
np.save("noisy.npy", noisy_matrix)